In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

model = load_model("emotion_model.hdf5",compile=False)
emotion_labels = ["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"]

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

masks = {
    "Happy" : cv2.imread("./masks/happy.png",cv2.IMREAD_UNCHANGED),
    "Sad" : cv2.imread("./masks/sad.png",cv2.IMREAD_UNCHANGED),
    "Angry" : cv2.imread("./masks/angry.png",cv2.IMREAD_UNCHANGED),
    "Neutral" : cv2.imread("./masks/neutral.png",cv2.IMREAD_UNCHANGED),
}

def overlay_mask(face_img,mask_img,x,y,w,h):
    mask_resized = cv2.resize(mask_img,(w,h))
    mask_rgb = mask_resized[:,:,:3]
    mask_alpha = mask_resized[:,:,3]/255.0

    for c in range(3):
        face_img[y:y+h,x:x+w,c] = ((1.0-mask_alpha)*face_img[y:y+h,x:x+w,c]+mask_alpha*mask_rgb[:,:,c])


cap = cv2.VideoCapture(0)

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break

    f = cv2.flip(f,1)
    gray = cv2.cvtColor(f,cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        face_roi = gray[y:y+h,x:x+w]
        face_resized = cv2.resize(face_roi,(64,64))
        face_resized = face_resized.astype("float")/255.0
        face_resized = img_to_array(face_resized)
        face_resized = np.expand_dims(face_resized,axis=-1)
        face_resized = np.expand_dims(face_resized,axis=0)

        preds = model.predict(face_resized,verbose=0)[0]
        emotion = emotion_labels[np.argmax(preds)]

        label = f"{emotion} ({int(np.max(preds)*100)}%)"
        cv2.putText(f,label,(x,y-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,0.9,(255,255,0),2)

        if emotion in ['Happy','Sad','Angry','Neutral'] and emotion in masks:
            overlay_mask(f,masks[emotion],x,y,w,h)

    cv2.imshow("Emotion",f)

    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
# Emotion Detector without mask
import pandas as pd
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

model = load_model("emotion_model.hdf5",compile=False)
emotion_labels = ["Angry","Disgust","Fear","Happy","Sad","Surprise","Neutral"] 

cap = cv2.VideoCapture(0)

mp_face = mp.solutions.face_detection
mp_draw = mp.solutions.drawing_utils
face = mp_face.FaceDetection(min_detection_confidence=0.7)

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,1)
    h,w,_ = f.shape

    res = face.process(rgb)
    
    if res.detections:
        for detection in res.detections:
            bboxc = detection.location_data.relative_bounding_box
            x = int(bboxc.xmin*w)
            y = int(bboxc.ymin*h)
            bw = int(bboxc.width*w)
            bh = int(bboxc.height*h)

            x,y = max(0,x),max(0,y)

            face_roi = f[y:y+bh,x:x+bw]
            if face_roi.shape[0]<64 or face_roi.shape[1]<64:
                continue

            gray_face = cv2.cvtColor(f,cv2.COLOR_BGR2GRAY)
            face_resized = cv2.resize(gray_face,(64,64))
            face_resized = face_resized.astype("float32")/255.0
            face_resized = np.expand_dims(face_resized,axis=-1)
            face_resized = np.expand_dims(face_resized,axis=0)

            preds = model.predict(face_resized,verbose=0)[0]
            emotion_idx = np.argmax(preds)
            emotion = emotion_labels[emotion_idx]
            confidence = int(np.max(preds)*100)

            colors = (0,255,0) if emotion == "Happy" else (0,0,255)
            cv2.rectangle(f,(x,y),(x+bw,y+bh),colors,2)
            label = f"{emotion} ({confidence}%)"
            cv2.putText(f,label,(x,y-10),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,colors,2)



    cv2.imshow("Emotion Detector",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()